### From MLG to nn

In [1]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
from torch.utils.data import DataLoader, Dataset
import torch
from torch import nn


In [2]:
df  = datasets.load_diabetes()

In [3]:
X = torch.from_numpy(df.data)
y = torch.from_numpy(df.target)

In [4]:
X

tensor([[ 0.0381,  0.0507,  0.0617,  ..., -0.0026,  0.0199, -0.0176],
        [-0.0019, -0.0446, -0.0515,  ..., -0.0395, -0.0683, -0.0922],
        [ 0.0853,  0.0507,  0.0445,  ..., -0.0026,  0.0029, -0.0259],
        ...,
        [ 0.0417,  0.0507, -0.0159,  ..., -0.0111, -0.0469,  0.0155],
        [-0.0455, -0.0446,  0.0391,  ...,  0.0266,  0.0445, -0.0259],
        [-0.0455, -0.0446, -0.0730,  ..., -0.0395, -0.0042,  0.0031]],
       dtype=torch.float64)

### Nothing new until now

In [5]:
from torch.utils.data import TensorDataset
dataset = TensorDataset(X, y)

from torch.utils.data import DataLoader


train_loader = DataLoader(dataset, batch_size = 25, shuffle=True)



In [6]:
#https://www.analyticsvidhya.com/blog/2021/08/linear-regression-and-gradient-descent-in-pytorch/
def rmse(y,y_hat):
    return torch.sqrt(((torch.pow(y-y_hat,2))/torch.numel(y)).sum())

loss =  

In [56]:
w = torch.randn(1,int(X[1].numel()), requires_grad=True, dtype=torch.float64)
#b = torch.randn(1,int(X.numel()/X[1].numel()), dtype=torch.float64)
#w = torch.randn(2, 3, requires_grad=True)
#b = torch.randn(2, requires_grad=True)
b = torch.randn(int(X.numel()/X[1].numel()), requires_grad=True)

b

tensor([ 1.3235e+00,  1.4424e+00,  1.0887e+00,  4.8829e-01, -8.6623e-01,
         1.2200e-01, -2.1710e-01,  8.7545e-01,  1.5057e+00,  4.6262e-01,
        -2.1117e-01,  8.7289e-01,  3.3605e-01, -2.6177e-01,  1.1483e+00,
         5.2294e-02, -2.5033e-01, -6.0430e-01, -1.2211e+00,  1.9518e-01,
        -4.6250e-01,  9.0810e-02,  2.1900e-01,  1.3171e+00, -6.5161e-01,
         1.6252e+00,  3.0523e-02,  5.8338e-01,  5.5603e-01, -2.2033e+00,
        -3.4900e-01, -6.6269e-02, -5.2196e-01,  1.4333e+00,  2.0604e-01,
         2.0538e-01, -6.2226e-01, -9.7405e-01, -3.5244e-01,  5.6247e-01,
         2.1188e-01, -5.7858e-01,  3.0509e+00, -6.5419e-01, -1.0861e+00,
         1.1145e+00, -4.7073e-01, -6.5196e-01,  1.1748e+00,  3.8939e-01,
         8.3013e-02,  5.9911e-01,  4.6505e-01,  1.2892e-01,  3.5219e-01,
        -1.8115e+00,  6.2478e-01, -1.0230e+00, -1.1161e+00,  1.2991e+00,
         1.1134e-02,  5.6615e-01, -1.2895e+00,  7.2954e-01,  1.3603e+00,
        -4.0245e-01, -1.9348e+00,  1.5938e+00,  1.0

In [59]:
def LinearReg(X):
    y_t = X @ w.transpose(0,1)  + b
    return y_t
    

In [60]:
#LinearReg(X)
loss = rmse(y,LinearReg(X))
loss.backward()

In [62]:

b.numel()


442

In [14]:
learning_rate = 0.01
for i in range(100):    
    y_hat = X @ w.transpose(0,1)  + b
    loss = rmse(y,y_hat)
    loss.backward()
    with torch.no_grad():
        w -= learning_rate * w.grad
        b -= learning_rate * b.grad
        w.grad = None
        b.grad = None
    if i == 50 or i==90:
        print(i)

TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

In [17]:
train_loader

TypeError: 'DataLoader' object is not subscriptable

In [60]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(df.data,df.target)

In [62]:
from sklearn.metrics import mean_squared_error

mean_squared_error(reg.predict(df.data),df.target)

2859.6903987680657

In [92]:
class LinearReg():
   def __init__(self):
      self.w = torch.randn(1,int(X[1].numel()), requires_grad=True)
      self.b = torch.randn(int(X.numel()/X[1].numel()), 1, requires_grad=True)
   def LinearReg(X):
      y = X.float() @ self.w.transpose(0,1) + self.b
      return self.y

   def rmse(self,y,y_hat):
      return torch.sqrt(((torch.pow(self.y-self.y_hat,2))/torch.numel(self.y)).sum())

   def fit(self,X):
      learning_rate = 0.01
      for i in range(300):
         y_hat = LinearReg(X)
         loss = rmse(self.y,y_hat)
         loss.backward()
         with torch.no_grad():
            w -= learning_rate * self.w.grad
            b -= learning_rate * self.b.grad
            self.w.grad = None
            self.b.grad = None


In [10]:
y

tensor([261.,  68., 191., 264., 127.,  63., 189., 257.,  88., 184., 252., 110.,
        178., 118., 146.,  83., 173., 248., 253., 111.,  71., 122., 150., 132.,
         97.], dtype=torch.float64)